In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import xgboost as xgb
import numpy as np 
import math 

from sklearn.metrics import accuracy_score
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display
from sklearn import tree

In [2]:
#load data
games = pd.read_csv('Data/games.csv', dtype = {'GAME_ID': str, 'HOME_TEAM_ID': str, 'VISITOR_TEAM_ID': str})
games.head(3)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0


In [3]:
games.dtypes

GAME_DATE_EST        object
GAME_ID              object
GAME_STATUS_TEXT     object
HOME_TEAM_ID         object
VISITOR_TEAM_ID      object
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

In [4]:
# games = games.loc[games['SEASON']>= 2014, :]

In [5]:
games["SEASON"].value_counts()

2014    1418
2015    1416
2016    1405
2017    1382
2018    1378
2019    1241
2020      49
Name: SEASON, dtype: int64

In [6]:
games.loc[games['HOME_TEAM_ID']==1610612753,:].sort_values(by='GAME_DATE_EST')

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS


In [7]:
games = games.dropna()

In [8]:
games = games.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [9]:
games = games.sort_values(by=["HOME_TEAM_ID", "GAME_DATE_EST"])
list(games.columns)

['GAME_DATE_EST',
 'GAME_ID',
 'GAME_STATUS_TEXT',
 'HOME_TEAM_ID',
 'VISITOR_TEAM_ID',
 'SEASON',
 'TEAM_ID_home',
 'PTS_home',
 'FG_PCT_home',
 'FT_PCT_home',
 'FG3_PCT_home',
 'AST_home',
 'REB_home',
 'TEAM_ID_away',
 'PTS_away',
 'FG_PCT_away',
 'FT_PCT_away',
 'FG3_PCT_away',
 'AST_away',
 'REB_away',
 'HOME_TEAM_WINS']

In [10]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
23515,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
23453,2014-10-18,11400077,Final,1610612737,1610612765,2014,1610612737,100.0,0.429,0.586,...,28.0,43.0,1610612765,104.0,0.469,0.645,0.381,28.0,47.0,0
23443,2014-10-20,11400089,Final,1610612737,1610612766,2014,1610612737,117.0,0.452,0.810,...,27.0,40.0,1610612766,114.0,0.446,0.828,0.364,31.0,52.0,1
23380,2014-11-01,21400032,Final,1610612737,1610612754,2014,1610612737,102.0,0.507,0.758,...,26.0,37.0,1610612754,92.0,0.383,0.857,0.375,25.0,44.0,1
23326,2014-11-08,21400084,Final,1610612737,1610612752,2014,1610612737,103.0,0.407,0.778,...,18.0,41.0,1610612752,96.0,0.476,0.727,0.381,26.0,44.0,1


In [11]:
games["FG_PCT_home_ROLLING"] = games.groupby(['HOME_TEAM_ID'])['FG_PCT_home'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FG_PCT_home_ROLLING"] = games.apply(lambda x: x['FG_PCT_home'] if math.isnan(x["FG_PCT_home_ROLLING"]) else x["FG_PCT_home_ROLLING"], axis=1)

games["FT_PCT_home_ROLLING"] = games.groupby(['HOME_TEAM_ID'])['FT_PCT_home'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FT_PCT_home_ROLLING"] = games.apply(lambda x: x['FT_PCT_home'] if math.isnan(x["FT_PCT_home_ROLLING"]) else x["FT_PCT_home_ROLLING"], axis=1)

games["FG3_PCT_home_ROLLING"] = games.groupby(['HOME_TEAM_ID'])['FG3_PCT_home'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FG3_PCT_home_ROLLING"] = games.apply(lambda x: x['FG3_PCT_home'] if math.isnan(x["FG3_PCT_home_ROLLING"]) else x["FG3_PCT_home_ROLLING"], axis=1)

games["AST_home_ROLLING"] = games.groupby(['HOME_TEAM_ID'])['AST_home'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["AST_home_ROLLING"] = games.apply(lambda x: x['AST_home'] if math.isnan(x["AST_home_ROLLING"]) else x["AST_home_ROLLING"], axis=1)

games["REB_home_ROLLING"] = games.groupby(['HOME_TEAM_ID'])['REB_home'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["REB_home_ROLLING"] = games.apply(lambda x: x['REB_home'] if math.isnan(x["REB_home_ROLLING"]) else x["REB_home_ROLLING"], axis=1)

games["FG_PCT_away_ROLLING"] = games.groupby(['VISITOR_TEAM_ID'])['FG_PCT_away'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FG_PCT_away_ROLLING"] = games.apply(lambda x: x['FG_PCT_away'] if math.isnan(x["FG_PCT_away_ROLLING"]) else x["FG_PCT_away_ROLLING"], axis=1)

games["FT_PCT_away_ROLLING"] = games.groupby(['VISITOR_TEAM_ID'])['FT_PCT_away'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FT_PCT_away_ROLLING"] = games.apply(lambda x: x['FT_PCT_away'] if math.isnan(x["FT_PCT_away_ROLLING"]) else x["FT_PCT_away_ROLLING"], axis=1)

games["FG3_PCT_away_ROLLING"] = games.groupby(['VISITOR_TEAM_ID'])['FG3_PCT_away'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["FG3_PCT_away_ROLLING"] = games.apply(lambda x: x['FG3_PCT_away'] if math.isnan(x["FG3_PCT_away_ROLLING"]) else x["FG3_PCT_away_ROLLING"], axis=1)

games["AST_away_ROLLING"] = games.groupby(['VISITOR_TEAM_ID'])['AST_away'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["AST_away_ROLLING"] = games.apply(lambda x: x['AST_away'] if math.isnan(x["AST_away_ROLLING"]) else x["AST_away_ROLLING"], axis=1)

games["REB_away_ROLLING"] = games.groupby(['VISITOR_TEAM_ID'])['REB_away'].apply(lambda x: x.rolling(5, 1).mean().shift())
games["REB_away_ROLLING"] = games.apply(lambda x: x['REB_away'] if math.isnan(x["REB_away_ROLLING"]) else x["REB_away_ROLLING"], axis=1)



In [12]:
games.head(10)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,FG_PCT_away_ROLLING,FT_PCT_away_ROLLING,FG3_PCT_away_ROLLING,AST_away_ROLLING,REB_away_ROLLING
23515,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,0.419000,0.82100,0.421000,24.000000,50.000000,0.366,0.643,0.375,17.0,43.0
23453,2014-10-18,11400077,Final,1610612737,1610612765,2014,1610612737,100.0,0.429,0.586,...,0.419000,0.82100,0.421000,24.000000,50.000000,0.469,0.645,0.381,28.0,47.0
23443,2014-10-20,11400089,Final,1610612737,1610612766,2014,1610612737,117.0,0.452,0.810,...,0.424000,0.70350,0.414000,26.000000,46.500000,0.446,0.828,0.364,31.0,52.0
23380,2014-11-01,21400032,Final,1610612737,1610612754,2014,1610612737,102.0,0.507,0.758,...,0.433333,0.73900,0.369333,26.333333,44.333333,0.383,0.857,0.375,25.0,44.0
23326,2014-11-08,21400084,Final,1610612737,1610612752,2014,1610612737,103.0,0.407,0.778,...,0.451750,0.74375,0.364500,26.250000,42.500000,0.476,0.727,0.381,26.0,44.0
23297,2014-11-12,21400110,Final,1610612737,1610612762,2014,1610612737,100.0,0.513,0.722,...,0.442800,0.75060,0.373400,24.600000,42.200000,0.500,0.500,0.217,28.0,30.0
23284,2014-11-14,21400124,Final,1610612737,1610612748,2014,1610612737,114.0,0.560,0.826,...,0.461600,0.73080,0.379200,24.400000,41.400000,0.473,0.920,0.476,27.0,32.0
23255,2014-11-18,21400155,Final,1610612737,1610612747,2014,1610612737,109.0,0.482,0.783,...,0.487800,0.77880,0.376400,25.400000,40.000000,0.540,0.636,0.353,24.0,44.0
23232,2014-11-21,21400174,Final,1610612737,1610612765,2014,1610612737,99.0,0.475,0.692,...,0.493800,0.77340,0.387000,24.400000,39.600000,0.469,0.645,0.381,28.0,47.0
23196,2014-11-26,21400215,Final,1610612737,1610612761,2014,1610612737,115.0,0.512,0.875,...,0.487400,0.76020,0.358600,23.200000,41.400000,0.512,0.848,0.483,24.0,36.0


In [24]:
counter = 0
home_var = {'GAME_ID': [],
           'FG_PCT_home': [],
           'FT_PCT_home': [],
           'FG3_PCT_home': [], 'AST_home': [], 'REB_home': []
           }
home_data = pd.DataFrame(home_var)
print(len(games))

for index,row in games.iterrows():
    temp_df = games.loc[(games['HOME_TEAM_ID']==row["HOME_TEAM_ID"]) & (games["GAME_DATE_EST"] < row["GAME_DATE_EST"]),:]

    agg_df = temp_df[['FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home','REB_home', 'HOME_TEAM_ID']].groupby("HOME_TEAM_ID").mean()
    agg_df["GAME_ID"] = row["GAME_ID"]
    home_data = home_data.append(agg_df)
    print(index)
    counter +=1
    print(counter)

        
home_data

8289
23515
1
23453
2
23443
3
23380
4
23326
5
23297
6
23284
7
23255
8
23232
9
23196
10
23181
11
23175
12
23152
13
23114
14
23091
15
23076
16
23054
17
22991
18
22973
19
22940
20
22883
21
22851
22
22793
23
22780
24
22763
25
22750
26
22727
27
22712
28
22703
29
22675
30
22662
31
22602
32
22563
33
22548
34
22522
35
22497
36
22470
37
22369
38
22331
39
22309
40
22269
41
22254
42
22228
43
22203
44
22176
45
22167
46
22146
47
22140
48
22136
49
22122
50
22115
51
22113
52
22053
53
22035
54
22017
55
21999
56
21975
57
21935
58
21911
59
21899
60
21883
61
21854
62
21832
63
21789
64
21743
65
21730
66
21717
67
21653
68
21639
69
21625
70
21587
71
21573
72
21554
73
21478
74
21447
75
21394
76
21383
77
21315
78
21276
79
21244
80
21226
81
21187
82
21180
83
21166
84
21138
85
21120
86
21024
87
21016
88
20986
89
20970
90
20956
91
20924
92
20868
93
20841
94
20825
95
20810
96
20770
97
20759
98
20738
99
20718
100
20714
101
20650
102
20634
103
20609
104
20573
105
20547
106
20530
107
20509
108
20479
109
20454
110
204

543
858
521
859
507
860
493
861
477
862
439
863
409
864
370
865
355
866
335
867
318
868
301
869
283
870
278
871
38
872
25
873
23503
874
23476
875
23465
876
23424
877
23410
878
23375
879
23362
880
23301
881
23288
882
23207
883
23156
884
23101
885
23082
886
23059
887
23048
888
23015
889
22976
890
22943
891
22923
892
22900
893
22868
894
22784
895
22749
896
22742
897
22730
898
22716
899
22694
900
22681
901
22651
902
22639
903
22622
904
22581
905
22569
906
22555
907
22513
908
22501
909
22484
910
22427
911
22413
912
22350
913
22337
914
22316
915
22256
916
22234
917
22189
918
22166
919
22159
920
22074
921
22038
922
22007
923
21966
924
21946
925
21924
926
21896
927
21843
928
21818
929
21803
930
21739
931
21719
932
21698
933
21665
934
21576
935
21553
936
21516
937
21473
938
21457
939
21406
940
21375
941
21359
942
21342
943
21332
944
21310
945
21293
946
21277
947
21256
948
21212
949
21190
950
21144
951
21127
952
21092
953
21076
954
21063
955
20980
956
20963
957
20948
958
20913
959
20901
960
2087

16204
1645
917
1646
891
1647
888
1648
842
1649
817
1650
763
1651
714
1652
699
1653
684
1654
668
1655
656
1656
646
1657
606
1658
572
1659
526
1660
496
1661
469
1662
430
1663
416
1664
375
1665
308
1666
294
1667
276
1668
257
1669
234
1670
218
1671
181
1672
171
1673
152
1674
115
1675
105
1676
93
1677
15
1678
23506
1679
23436
1680
23405
1681
23369
1682
23342
1683
23302
1684
23253
1685
23239
1686
23208
1687
23188
1688
23157
1689
23097
1690
23065
1691
23044
1692
23029
1693
23019
1694
22979
1695
22959
1696
22945
1697
22916
1698
22887
1699
22837
1700
22812
1701
22790
1702
22770
1703
22755
1704
22708
1705
22641
1706
22582
1707
22570
1708
22556
1709
22538
1710
22525
1711
22491
1712
22474
1713
22459
1714
22445
1715
22351
1716
22339
1717
22275
1718
22250
1719
22236
1720
22215
1721
22079
1722
22046
1723
21978
1724
21931
1725
21900
1726
21887
1727
21874
1728
21821
1729
21805
1730
21791
1731
21772
1732
21688
1733
21666
1734
21644
1735
21599
1736
21583
1737
21533
1738
21495
1739
21443
1740
21422
1741
2

19179
2417
19165
2418
19114
2419
19070
2420
19045
2421
19027
2422
19009
2423
18951
2424
18930
2425
18918
2426
18904
2427
18816
2428
18807
2429
18793
2430
18779
2431
18765
2432
18751
2433
18736
2434
18670
2435
18644
2436
18603
2437
18547
2438
18534
2439
18518
2440
18475
2441
18463
2442
18392
2443
18376
2444
18355
2445
18333
2446
18273
2447
18208
2448
18187
2449
18136
2450
18121
2451
18116
2452
18099
2453
18077
2454
18048
2455
18033
2456
18018
2457
17976
2458
17967
2459
17944
2460
17935
2461
17930
2462
17918
2463
17915
2464
17913
2465
17907
2466
17903
2467
17885
2468
17877
2469
17826
2470
17776
2471
17763
2472
17735
2473
17645
2474
17618
2475
17607
2476
17575
2477
17521
2478
17501
2479
17429
2480
17414
2481
17385
2482
17373
2483
17352
2484
17333
2485
17317
2486
17287
2487
17237
2488
17220
2489
17208
2490
17184
2491
17171
2492
17154
2493
17108
2494
17093
2495
17082
2496
17007
2497
16991
2498
16932
2499
16909
2500
16852
2501
16830
2502
16818
2503
16804
2504
16790
2505
16751
2506
16708
2507

21538
3215
21509
3216
21486
3217
21467
3218
21374
3219
21284
3220
21233
3221
21215
3222
21181
3223
21164
3224
21151
3225
21107
3226
21091
3227
21071
3228
21007
3229
20985
3230
20971
3231
20923
3232
20896
3233
20840
3234
20826
3235
20806
3236
20765
3237
20756
3238
20734
3239
20726
3240
20716
3241
20712
3242
20707
3243
20659
3244
20644
3245
20623
3246
20610
3247
20591
3248
20569
3249
20551
3250
20539
3251
20471
3252
20458
3253
20434
3254
20418
3255
20349
3256
20334
3257
20272
3258
20226
3259
20211
3260
20201
3261
20184
3262
20167
3263
20153
3264
20117
3265
20080
3266
19966
3267
19952
3268
19936
3269
19921
3270
19882
3271
19868
3272
19853
3273
19829
3274
19763
3275
19720
3276
19688
3277
19666
3278
19634
3279
19613
3280
19581
3281
19569
3282
19553
3283
19535
3284
19521
3285
19462
3286
19446
3287
19381
3288
19364
3289
19277
3290
19237
3291
19225
3292
19176
3293
19164
3294
19151
3295
19128
3296
19112
3297
19100
3298
18998
3299
18972
3300
18949
3301
18889
3302
18874
3303
18803
3304
18778
3305

...

16715
5509
16697
5510
16681
5511
16666
5512
16650
5513
16509
5514
16480
5515
16452
5516
16430
5517
16413
5518
16364
5519
16343
5520
16322
5521
16308
5522
16294
5523
16265
5524
16243
5525
16200
5526
16187
5527
880
5528
866
5529
838
5530
831
5531
794
5532
778
5533
707
5534
690
5535
676
5536
654
5537
641
5538
584
5539
569
5540
505
5541
455
5542
448
5543
417
5544
361
5545
347
5546
339
5547
315
5548
295
5549
274
5550
222
5551
207
5552
180
5553
155
5554
140
5555
21
5556
10
5557
23500
5558
23486
5559
23409
5560
23374
5561
23363
5562
23344
5563
23322
5564
23309
5565
23278
5566
23266
5567
23241
5568
23189
5569
23171
5570
23139
5571
23057
5572
23042
5573
22980
5574
22958
5575
22947
5576
22917
5577
22902
5578
22878
5579
22864
5580
22839
5581
22801
5582
22775
5583
22761
5584
22688
5585
22673
5586
22626
5587
22593
5588
22573
5589
22559
5590
22505
5591
22462
5592
22447
5593
22360
5594
22327
5595
22313
5596
22301
5597
22277
5598
22253
5599
22225
5600
22160
5601
22152
5602
22094
5603
22080
5604
22036


18085
6280
18058
6281
18020
6282
18005
6283
17964
6284
17952
6285
17892
6286
17874
6287
17863
6288
17827
6289
17777
6290
17759
6291
17745
6292
17706
6293
17697
6294
17656
6295
17599
6296
17579
6297
17511
6298
17496
6299
17463
6300
17445
6301
17430
6302
17415
6303
17402
6304
17388
6305
17361
6306
17330
6307
17288
6308
17268
6309
17255
6310
17215
6311
17185
6312
17145
6313
17096
6314
17083
6315
17067
6316
17053
6317
16918
6318
16895
6319
16877
6320
16840
6321
16806
6322
16797
6323
16780
6324
16766
6325
16711
6326
16685
6327
16670
6328
16610
6329
16585
6330
16578
6331
16564
6332
16518
6333
16483
6334
16460
6335
16450
6336
16428
6337
16412
6338
16369
6339
16342
6340
16325
6341
16313
6342
16280
6343
16215
6344
16199
6345
16186
6346
924
6347
904
6348
856
6349
808
6350
793
6351
749
6352
727
6353
715
6354
685
6355
605
6356
587
6357
555
6358
534
6359
516
6360
497
6361
360
6362
338
6363
324
6364
263
6365
240
6366
217
6367
186
6368
172
6369
149
6370
39
6371
23477
6372
23460
6373
23447
6374
23434


19885
7069
19857
7070
19831
7071
19782
7072
19768
7073
19752
7074
19693
7075
19678
7076
19655
7077
19598
7078
19532
7079
19494
7080
19464
7081
19434
7082
19409
7083
19370
7084
19340
7085
19333
7086
19319
7087
19304
7088
19300
7089
19256
7090
19205
7091
19183
7092
19132
7093
19117
7094
19105
7095
19090
7096
19060
7097
19041
7098
19034
7099
19016
7100
18957
7101
18933
7102
18912
7103
18893
7104
18872
7105
18848
7106
18745
7107
18727
7108
18679
7109
18656
7110
18572
7111
18543
7112
18536
7113
18466
7114
18393
7115
18370
7116
18358
7117
18338
7118
18327
7119
18314
7120
18285
7121
18261
7122
18205
7123
18191
7124
18175
7125
18152
7126
18093
7127
18080
7128
18053
7129
18035
7130
17997
7131
17958
7132
17950
7133
17940
7134
17927
7135
17923
7136
17886
7137
17815
7138
17790
7139
17714
7140
17691
7141
17678
7142
17665
7143
17580
7144
17538
7145
17483
7146
17468
7147
17426
7148
17376
7149
17354
7150
17336
7151
17320
7152
17303
7153
17224
7154
17210
7155
17203
7156
17186
7157
17160
7158
17141
7159

21070
7846
20995
7847
20977
7848
20967
7849
20955
7850
20938
7851
20922
7852
20914
7853
20893
7854
20866
7855
20819
7856
20787
7857
20750
7858
20745
7859
20651
7860
20614
7861
20602
7862
20567
7863
20552
7864
20540
7865
20507
7866
20440
7867
20402
7868
20387
7869
20372
7870
20356
7871
20288
7872
20273
7873
20231
7874
20187
7875
20157
7876
20140
7877
20123
7878
20109
7879
20065
7880
20050
7881
19962
7882
19932
7883
19922
7884
19854
7885
19838
7886
19817
7887
19801
7888
19785
7889
19748
7890
19736
7891
19714
7892
19697
7893
19650
7894
19627
7895
19609
7896
19582
7897
19567
7898
19550
7899
19485
7900
19466
7901
19419
7902
19384
7903
19264
7904
19251
7905
19233
7906
19198
7907
19163
7908
19148
7909
19081
7910
19077
7911
19048
7912
19035
7913
19021
7914
18961
7915
18899
7916
18838
7917
18822
7918
18809
7919
18772
7920
18732
7921
18706
7922
18675
7923
18628
7924
18563
7925
18540
7926
18527
7927
18503
7928
18481
7929
18462
7930
18446
7931
18429
7932
18415
7933
18402
7934
18387
7935
18366
7936

,GAME_ID,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home
1610612737,11400077,0.419000,0.821000,0.421000,24.000000,50.000000
1610612737,11400089,0.424000,0.703500,0.414000,26.000000,46.500000
1610612737,21400032,0.433333,0.739000,0.369333,26.333333,44.333333
1610612737,21400084,0.451750,0.743750,0.364500,26.250000,42.500000
1610612737,21400110,0.442800,0.750600,0.373400,24.600000,42.200000
...,...,...,...,...,...,...
1610612766,21900909,0.443406,0.772815,0.345591,22.649606,44.594488
1610612766,21900926,0.443533,0.772616,0.345859,22.658824,44.600000
1610612766,21900940,0.443730,0.772805,0.346047,22.664062,44.558594
1610612766,12000006,0.444062,0.772755,0.346525,22.688716,44.501946


In [26]:
counter = 0
away_var = {'GAME_ID': [],
           'FG_PCT_away': [],
           'FT_PCT_away': [],
           'FG3_PCT_away': [], 'AST_away': [], 'REB_away': []
           }
away_data = pd.DataFrame(away_var)


for index,row in games.iterrows():
    temp_df = games.loc[(games['VISITOR_TEAM_ID']==row["VISITOR_TEAM_ID"]) & (games["GAME_DATE_EST"] < row["GAME_DATE_EST"]),:]

    agg_df = temp_df[['FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away','REB_away', 'VISITOR_TEAM_ID']].groupby("VISITOR_TEAM_ID").mean()
    agg_df["GAME_ID"] = row["GAME_ID"]
    away_data = away_data.append(agg_df)
    print(index)
    counter +=1
    print(counter)


23515
1
23453
2
23443
3
23380
4
23326
5
23297
6
23284
7
23255
8
23232
9
23196
10
23181
11
23175
12
23152
13
23114
14
23091
15
23076
16
23054
17
22991
18
22973
19
22940
20
22883
21
22851
22
22793
23
22780
24
22763
25
22750
26
22727
27
22712
28
22703
29
22675
30
22662
31
22602
32
22563
33
22548
34
22522
35
22497
36
22470
37
22369
38
22331
39
22309
40
22269
41
22254
42
22228
43
22203
44
22176
45
22167
46
22146
47
22140
48
22136
49
22122
50
22115
51
22113
52
22053
53
22035
54
22017
55
21999
56
21975
57
21935
58
21911
59
21899
60
21883
61
21854
62
21832
63
21789
64
21743
65
21730
66
21717
67
21653
68
21639
69
21625
70
21587
71
21573
72
21554
73
21478
74
21447
75
21394
76
21383
77
21315
78
21276
79
21244
80
21226
81
21187
82
21180
83
21166
84
21138
85
21120
86
21024
87
21016
88
20986
89
20970
90
20956
91
20924
92
20868
93
20841
94
20825
95
20810
96
20770
97
20759
98
20738
99
20718
100
20714
101
20650
102
20634
103
20609
104
20573
105
20547
106
20530
107
20509
108
20479
109
20454
110
20427
11

370
865
355
866
335
867
318
868
301
869
283
870
278
871
38
872
25
873
23503
874
23476
875
23465
876
23424
877
23410
878
23375
879
23362
880
23301
881
23288
882
23207
883
23156
884
23101
885
23082
886
23059
887
23048
888
23015
889
22976
890
22943
891
22923
892
22900
893
22868
894
22784
895
22749
896
22742
897
22730
898
22716
899
22694
900
22681
901
22651
902
22639
903
22622
904
22581
905
22569
906
22555
907
22513
908
22501
909
22484
910
22427
911
22413
912
22350
913
22337
914
22316
915
22256
916
22234
917
22189
918
22166
919
22159
920
22074
921
22038
922
22007
923
21966
924
21946
925
21924
926
21896
927
21843
928
21818
929
21803
930
21739
931
21719
932
21698
933
21665
934
21576
935
21553
936
21516
937
21473
938
21457
939
21406
940
21375
941
21359
942
21342
943
21332
944
21310
945
21293
946
21277
947
21256
948
21212
949
21190
950
21144
951
21127
952
21092
953
21076
954
21063
955
20980
956
20963
957
20948
958
20913
959
20901
960
20878
961
20821
962
20809
963
20796
964
20685
965
20675
966


496
1661
469
1662
430
1663
416
1664
375
1665
308
1666
294
1667
276
1668
257
1669
234
1670
218
1671
181
1672
171
1673
152
1674
115
1675
105
1676
93
1677
15
1678
23506
1679
23436
1680
23405
1681
23369
1682
23342
1683
23302
1684
23253
1685
23239
1686
23208
1687
23188
1688
23157
1689
23097
1690
23065
1691
23044
1692
23029
1693
23019
1694
22979
1695
22959
1696
22945
1697
22916
1698
22887
1699
22837
1700
22812
1701
22790
1702
22770
1703
22755
1704
22708
1705
22641
1706
22582
1707
22570
1708
22556
1709
22538
1710
22525
1711
22491
1712
22474
1713
22459
1714
22445
1715
22351
1716
22339
1717
22275
1718
22250
1719
22236
1720
22215
1721
22079
1722
22046
1723
21978
1724
21931
1725
21900
1726
21887
1727
21874
1728
21821
1729
21805
1730
21791
1731
21772
1732
21688
1733
21666
1734
21644
1735
21599
1736
21583
1737
21533
1738
21495
1739
21443
1740
21422
1741
21408
1742
21391
1743
21376
1744
21360
1745
21346
1746
21333
1747
21280
1748
21251
1749
21172
1750
21160
1751
21116
1752
21104
1753
21089
1754
2106

18121
2451
18116
2452
18099
2453
18077
2454
18048
2455
18033
2456
18018
2457
17976
2458
17967
2459
17944
2460
17935
2461
17930
2462
17918
2463
17915
2464
17913
2465
17907
2466
17903
2467
17885
2468
17877
2469
17826
2470
17776
2471
17763
2472
17735
2473
17645
2474
17618
2475
17607
2476
17575
2477
17521
2478
17501
2479
17429
2480
17414
2481
17385
2482
17373
2483
17352
2484
17333
2485
17317
2486
17287
2487
17237
2488
17220
2489
17208
2490
17184
2491
17171
2492
17154
2493
17108
2494
17093
2495
17082
2496
17007
2497
16991
2498
16932
2499
16909
2500
16852
2501
16830
2502
16818
2503
16804
2504
16790
2505
16751
2506
16708
2507
16691
2508
16645
2509
16631
2510
16598
2511
16589
2512
16565
2513
16550
2514
16547
2515
16539
2516
16496
2517
16469
2518
16441
2519
16427
2520
16409
2521
16349
2522
16299
2523
16284
2524
16263
2525
16217
2526
16196
2527
16175
2528
894
2529
877
2530
844
2531
827
2532
746
2533
726
2534
706
2535
642
2536
619
2537
596
2538
579
2539
568
2540
502
2541
487
2542
473
2543
438
254

20840
3234
20826
3235
20806
3236
20765
3237
20756
3238
20734
3239
20726
3240
20716
3241
20712
3242
20707
3243
20659
3244
20644
3245
20623
3246
20610
3247
20591
3248
20569
3249
20551
3250
20539
3251
20471
3252
20458
3253
20434
3254
20418
3255
20349
3256
20334
3257
20272
3258
20226
3259
20211
3260
20201
3261
20184
3262
20167
3263
20153
3264
20117
3265
20080
3266
19966
3267
19952
3268
19936
3269
19921
3270
19882
3271
19868
3272
19853
3273
19829
3274
19763
3275
19720
3276
19688
3277
19666
3278
19634
3279
19613
3280
19581
3281
19569
3282
19553
3283
19535
3284
19521
3285
19462
3286
19446
3287
19381
3288
19364
3289
19277
3290
19237
3291
19225
3292
19176
3293
19164
3294
19151
3295
19128
3296
19112
3297
19100
3298
18998
3299
18972
3300
18949
3301
18889
3302
18874
3303
18803
3304
18778
3305
18735
3306
18725
3307
18708
3308
18684
3309
18650
3310
18636
3311
18621
3312
18574
3313
18498
3314
18484
3315
18417
3316
18403
3317
18391
3318
18322
3319
18302
3320
18287
3321
18272
3322
18258
3323
18237
3324

...

22433
4791
22415
4792
22378
4793
22361
4794
22321
4795
22278
4796
22263
4797
22216
4798
22198
4799
22100
4800
22077
4801
22056
4802
22010
4803
21982
4804
21964
4805
21933
4806
21923
4807
21897
4808
21865
4809
21808
4810
21768
4811
21685
4812
21659
4813
21636
4814
21620
4815
21605
4816
21569
4817
21535
4818
21513
4819
21500
4820
21426
4821
21401
4822
21320
4823
21306
4824
21292
4825
21274
4826
21236
4827
21222
4828
21203
4829
21149
4830
21134
4831
21122
4832
21061
4833
21000
4834
20983
4835
20968
4836
20953
4837
20930
4838
20907
4839
20890
4840
20876
4841
20830
4842
20805
4843
20786
4844
20754
4845
20746
4846
20730
4847
20665
4848
20641
4849
20578
4850
20538
4851
20508
4852
20478
4853
20452
4854
20438
4855
20424
4856
20408
4857
20386
4858
20370
4859
20360
4860
20342
4861
20238
4862
20224
4863
20174
4864
20150
4865
20090
4866
20081
4867
20048
4868
20033
4869
19973
4870
19925
4871
19888
4872
19875
4873
19828
4874
19814
4875
19798
4876
19776
4877
19761
4878
19741
4879
19726
4880
19639
4881

23139
5571
23057
5572
23042
5573
22980
5574
22958
5575
22947
5576
22917
5577
22902
5578
22878
5579
22864
5580
22839
5581
22801
5582
22775
5583
22761
5584
22688
5585
22673
5586
22626
5587
22593
5588
22573
5589
22559
5590
22505
5591
22462
5592
22447
5593
22360
5594
22327
5595
22313
5596
22301
5597
22277
5598
22253
5599
22225
5600
22160
5601
22152
5602
22094
5603
22080
5604
22036
5605
21996
5606
21968
5607
21932
5608
21908
5609
21889
5610
21822
5611
21792
5612
21762
5613
21740
5614
21724
5615
21658
5616
21645
5617
21561
5618
21527
5619
21490
5620
21477
5621
21461
5622
21444
5623
21423
5624
21373
5625
21347
5626
21326
5627
21304
5628
21288
5629
21272
5630
21257
5631
21214
5632
21194
5633
21179
5634
21162
5635
21146
5636
21057
5637
21029
5638
20945
5639
20920
5640
20904
5641
20880
5642
20864
5643
20836
5644
20813
5645
20785
5646
20749
5647
20741
5648
20732
5649
20717
5650
20713
5651
20662
5652
20619
5653
20588
5654
20576
5655
20544
5656
20488
5657
20469
5658
20450
5659
20436
5660
20363
5661

793
6351
749
6352
727
6353
715
6354
685
6355
605
6356
587
6357
555
6358
534
6359
516
6360
497
6361
360
6362
338
6363
324
6364
263
6365
240
6366
217
6367
186
6368
172
6369
149
6370
39
6371
23477
6372
23460
6373
23447
6374
23434
6375
23384
6376
23339
6377
23318
6378
23289
6379
23270
6380
23236
6381
23219
6382
23200
6383
23185
6384
23102
6385
23086
6386
23063
6387
23013
6388
22995
6389
22977
6390
22938
6391
22924
6392
22869
6393
22820
6394
22743
6395
22695
6396
22667
6397
22642
6398
22623
6399
22613
6400
22591
6401
22576
6402
22514
6403
22482
6404
22458
6405
22443
6406
22425
6407
22405
6408
22389
6409
22368
6410
22357
6411
22299
6412
22261
6413
22257
6414
22235
6415
22207
6416
22057
6417
22034
6418
21994
6419
21962
6420
21939
6421
21907
6422
21872
6423
21855
6424
21834
6425
21819
6426
21804
6427
21784
6428
21771
6429
21703
6430
21672
6431
21649
6432
21627
6433
21600
6434
21563
6435
21547
6436
21532
6437
21517
6438
21488
6439
21476
6440
21421
6441
21402
6442
21389
6443
21370
6444
21300
644

18080
7128
18053
7129
18035
7130
17997
7131
17958
7132
17950
7133
17940
7134
17927
7135
17923
7136
17886
7137
17815
7138
17790
7139
17714
7140
17691
7141
17678
7142
17665
7143
17580
7144
17538
7145
17483
7146
17468
7147
17426
7148
17376
7149
17354
7150
17336
7151
17320
7152
17303
7153
17224
7154
17210
7155
17203
7156
17186
7157
17160
7158
17141
7159
17127
7160
17113
7161
17062
7162
17057
7163
17027
7164
17000
7165
16951
7166
16920
7167
16898
7168
16879
7169
16831
7170
16812
7171
16799
7172
16729
7173
16716
7174
16702
7175
16680
7176
16649
7177
16622
7178
16580
7179
16572
7180
16479
7181
16471
7182
16451
7183
16429
7184
16399
7185
16352
7186
16338
7187
16309
7188
16266
7189
16240
7190
16231
7191
918
7192
895
7193
881
7194
852
7195
823
7196
765
7197
733
7198
670
7199
655
7200
599
7201
585
7202
541
7203
532
7204
470
7205
457
7206
418
7207
398
7208
391
7209
376
7210
362
7211
348
7212
269
7213
249
7214
239
7215
201
7216
184
7217
156
7218
141
7219
113
7220
107
7221
94
7222
43
7223
30
7224
23

19198
7907
19163
7908
19148
7909
19081
7910
19077
7911
19048
7912
19035
7913
19021
7914
18961
7915
18899
7916
18838
7917
18822
7918
18809
7919
18772
7920
18732
7921
18706
7922
18675
7923
18628
7924
18563
7925
18540
7926
18527
7927
18503
7928
18481
7929
18462
7930
18446
7931
18429
7932
18415
7933
18402
7934
18387
7935
18366
7936
18348
7937
18329
7938
18291
7939
18242
7940
18226
7941
18119
7942
18105
7943
18083
7944
18037
7945
18021
7946
17999
7947
17858
7948
17845
7949
17821
7950
17782
7951
17767
7952
17751
7953
17685
7954
17644
7955
17587
7956
17557
7957
17541
7958
17528
7959
17506
7960
17498
7961
17484
7962
17457
7963
17443
7964
17400
7965
17383
7966
17337
7967
17322
7968
17251
7969
17235
7970
17169
7971
17157
7972
17148
7973
17079
7974
17064
7975
17047
7976
17038
7977
17009
7978
16987
7979
16928
7980
16887
7981
16860
7982
16833
7983
16800
7984
16787
7985
16704
7986
16692
7987
16657
7988
16628
7989
16615
7990
16579
7991
16571
7992
16511
7993
16501
7994
16489
7995
16440
7996
16422
7997

In [27]:
home_data= home_data.rename(columns = {'FG_PCT_home': 'FG_PCT_home_ALL', 'FT_PCT_home': 'FT_PCT_home_ALL', 'FG3_PCT_home': 'FG3_PCT_home_ALL', 'AST_home': 'AST_home_ALL','REB_home': 'REB_home_ALL'})
home_data.head()

,GAME_ID,FG_PCT_home_ALL,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL
1610612737,11400077,0.419000,0.82100,0.421000,24.000000,50.000000
1610612737,11400089,0.424000,0.70350,0.414000,26.000000,46.500000
1610612737,21400032,0.433333,0.73900,0.369333,26.333333,44.333333
1610612737,21400084,0.451750,0.74375,0.364500,26.250000,42.500000
1610612737,21400110,0.442800,0.75060,0.373400,24.600000,42.200000


In [28]:
away_data= away_data.rename(columns = {'FG_PCT_away': 'FG_PCT_away_ALL', 'FT_PCT_away': 'FT_PCT_away_ALL', 'FG3_PCT_away': 'FG3_PCT_away_ALL', 'AST_away': 'AST_away_ALL','REB_away': 'REB_away_ALL'})
away_data.head()

,GAME_ID,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL
1610612740,11400007,0.462000,0.706000,0.438000,19.000000,42.000000
1610612765,11400077,0.438000,0.600667,0.315333,24.000000,45.333333
1610612766,11400089,0.416667,0.802000,0.369333,21.666667,45.666667
1610612754,21400032,0.440000,0.711250,0.371000,20.500000,39.750000
1610612752,21400084,0.439857,0.781429,0.354857,21.857143,40.428571


In [29]:
clean_data = pd.merge(home_data, away_data, how="inner", on=["GAME_ID"])
final_data = pd.merge(games, clean_data, how="inner", on=["GAME_ID"])

In [30]:
final_data = final_data.dropna()

In [31]:
final_data = final_data.drop(columns = ["GAME_DATE_EST","GAME_STATUS_TEXT","TEAM_ID_home","PTS_home","TEAM_ID_away","PTS_away", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home", "AST_home", "REB_home", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away", "AST_away", "REB_away"])
final_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,...,FG_PCT_home_ALL,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL
0,11400077,1610612737,1610612765,2014,0,0.419000,0.82100,0.421000,24.000000,50.000000,...,0.419000,0.82100,0.421000,24.000000,50.000000,0.438000,0.600667,0.315333,24.000000,45.333333
1,11400089,1610612737,1610612766,2014,1,0.424000,0.70350,0.414000,26.000000,46.500000,...,0.424000,0.70350,0.414000,26.000000,46.500000,0.416667,0.802000,0.369333,21.666667,45.666667
2,21400032,1610612737,1610612754,2014,1,0.433333,0.73900,0.369333,26.333333,44.333333,...,0.433333,0.73900,0.369333,26.333333,44.333333,0.440000,0.711250,0.371000,20.500000,39.750000
3,21400084,1610612737,1610612752,2014,1,0.451750,0.74375,0.364500,26.250000,42.500000,...,0.451750,0.74375,0.364500,26.250000,42.500000,0.439857,0.781429,0.354857,21.857143,40.428571
4,21400110,1610612737,1610612762,2014,1,0.442800,0.75060,0.373400,24.600000,42.200000,...,0.442800,0.75060,0.373400,24.600000,42.200000,0.463889,0.749000,0.353556,21.888889,42.666667


In [32]:
train_data = final_data.loc[final_data['SEASON']!= 2020]
train_data["SEASON"].unique()

array([2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [33]:
test_data = final_data.loc[final_data['SEASON'] == 2020]
test_data["SEASON"].unique()

array([2020], dtype=int64)

In [34]:
test_data = test_data.drop(columns = ['SEASON'])
test_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,FG_PCT_away_ROLLING,...,FG_PCT_home_ALL,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL
269,12000001,1610612737,1610612753,0,0.4782,0.7872,0.4004,29.0,47.4,0.4396,...,0.459219,0.771733,0.353589,25.781481,43.540741,0.446535,0.758665,0.343747,23.096654,42.479554
270,12000013,1610612737,1610612753,1,0.4492,0.8064,0.3656,26.6,49.2,0.4300,...,0.458889,0.771945,0.353439,25.760148,43.605166,0.446574,0.758411,0.343533,23.107407,42.503704
567,12000041,1610612738,1610612751,0,0.4550,0.8282,0.3286,24.4,46.2,0.4264,...,0.453574,0.788007,0.354641,24.963087,44.852349,0.445095,0.766844,0.341650,22.657795,43.992395
866,12000007,1610612739,1610612754,1,0.4784,0.7334,0.3590,28.0,44.2,0.5328,...,0.467810,0.757083,0.371663,23.386667,43.793333,0.457529,0.778964,0.359598,22.807971,42.286232
867,12000019,1610612739,1610612754,1,0.4808,0.6884,0.3756,26.0,46.6,0.5200,...,0.467897,0.756645,0.371458,23.375415,43.807309,0.457412,0.778679,0.359531,22.819495,42.270758


In [35]:
train_data = train_data.drop(columns = ['SEASON'])
train_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,FG_PCT_away_ROLLING,...,FG_PCT_home_ALL,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL
0,11400077,1610612737,1610612765,0,0.419000,0.82100,0.421000,24.000000,50.000000,0.469,...,0.419000,0.82100,0.421000,24.000000,50.000000,0.438000,0.600667,0.315333,24.000000,45.333333
1,11400089,1610612737,1610612766,1,0.424000,0.70350,0.414000,26.000000,46.500000,0.446,...,0.424000,0.70350,0.414000,26.000000,46.500000,0.416667,0.802000,0.369333,21.666667,45.666667
2,21400032,1610612737,1610612754,1,0.433333,0.73900,0.369333,26.333333,44.333333,0.383,...,0.433333,0.73900,0.369333,26.333333,44.333333,0.440000,0.711250,0.371000,20.500000,39.750000
3,21400084,1610612737,1610612752,1,0.451750,0.74375,0.364500,26.250000,42.500000,0.476,...,0.451750,0.74375,0.364500,26.250000,42.500000,0.439857,0.781429,0.354857,21.857143,40.428571
4,21400110,1610612737,1610612762,1,0.442800,0.75060,0.373400,24.600000,42.200000,0.500,...,0.442800,0.75060,0.373400,24.600000,42.200000,0.463889,0.749000,0.353556,21.888889,42.666667


In [36]:
# ---------------------- MODELS --------------------# 

In [37]:
# Logistical Regression

In [38]:
# Assign X (data) and y (target)
X_train = train_data.drop("HOME_TEAM_WINS", axis=1)
y_train = train_data["HOME_TEAM_WINS"].values.reshape(-1, 1)
print(X_train.shape, y_train.shape)

(8198, 23) (8198, 1)


In [39]:
# Assign X (data) and y (target)
X_test = test_data.drop("HOME_TEAM_WINS", axis=1)
y_test = test_data["HOME_TEAM_WINS"].values.reshape(-1, 1)
print(X_test.shape, y_test.shape)

(49, 23) (49, 1)


In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


# X_train.head()

In [40]:
# Create the scalers using only the training data!
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [41]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# create your model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [43]:
# Fit (train) or model using the training data
classifier.fit(X_train_scaled, y_train)

C:\Users\sjrya\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [44]:
# Validate the model using the .score() method
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.6092949499878019
Testing Data Score: 0.40816326530612246


In [45]:
# XGBOOST

In [46]:
# create your model
model = XGBClassifier()

In [47]:
model.fit(X_train_scaled, y_train)

C:\Users\sjrya\anaconda3\envs\PythonData\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\sjrya\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[21:00:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
model.predict(X_test_scaled)

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1], dtype=int64)

In [49]:
y_pred= classifier.predict(X_test_scaled)

In [50]:
accuracy_score(y_test, y_pred)

0.40816326530612246

In [96]:
test_data['Predictions'] = model.predict(X_test_scaled)

In [97]:
test_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,FG_PCT_away_ROLLING,...,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL,Predictions
269,12000001,1610612737,1610612753,0,0.4782,0.7872,0.4004,29.0,47.4,0.4396,...,0.771733,0.353589,25.781481,43.540741,0.446535,0.758665,0.343747,23.096654,42.479554,1
270,12000013,1610612737,1610612753,1,0.4492,0.8064,0.3656,26.6,49.2,0.4300,...,0.771945,0.353439,25.760148,43.605166,0.446574,0.758411,0.343533,23.107407,42.503704,1
567,12000041,1610612738,1610612751,0,0.4550,0.8282,0.3286,24.4,46.2,0.4264,...,0.788007,0.354641,24.963087,44.852349,0.445095,0.766844,0.341650,22.657795,43.992395,0
866,12000007,1610612739,1610612754,1,0.4784,0.7334,0.3590,28.0,44.2,0.5328,...,0.757083,0.371663,23.386667,43.793333,0.457529,0.778964,0.359598,22.807971,42.286232,1
867,12000019,1610612739,1610612754,1,0.4808,0.6884,0.3756,26.0,46.6,0.5200,...,0.756645,0.371458,23.375415,43.807309,0.457412,0.778679,0.359531,22.819495,42.270758,0


In [131]:

#load data
games_untouched = pd.read_csv('Data/games.csv', dtype = {'GAME_ID': str, 'HOME_TEAM_ID': str, 'VISITOR_TEAM_ID': str})
games_untouched.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0
3,2020-12-18,12000039,Final,1610612754,1610612755,2020,1610612754,107.0,0.371,0.692,...,19.0,45.0,1610612755,113.0,0.533,0.629,0.355,23.0,48.0,0
4,2020-12-18,12000040,Final,1610612761,1610612748,2020,1610612761,105.0,0.380,0.737,...,27.0,37.0,1610612748,117.0,0.534,0.741,0.514,30.0,51.0,0


In [146]:
def lookup(game_id):
    game_lookup = games_untouched.loc[games_untouched["GAME_ID"]==game_id, ['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST']]
    print(game_lookup["HOME_TEAM_ID"])
    home_name = teams.loc[teams["TEAM_ID"] ==game_lookup['HOME_TEAM_ID'][0], ["NICKNAME"]].iloc[0]
    away_name = teams.loc[teams["TEAM_ID"] ==game_lookup['VISITOR_TEAM_ID'][0], ["NICKNAME"]].iloc[0]
    date = game_lookup['GAME_DATE_EST']

    print(home_name)
    print(away_name)
    print(date)
lookup('12000047')

0    1610612753
Name: HOME_TEAM_ID, dtype: object
NICKNAME    Magic
Name: 14, dtype: object
NICKNAME    Hornets
Name: 27, dtype: object
0    2020-12-19
Name: GAME_DATE_EST, dtype: object


In [133]:
# Google 

In [125]:
def moneyback(won, odds):
    if not won:
        return -100
    if odds > 0:
        return 100 + odds
    else:
        return 100 + ((100/abs(odds)*100))
                      
moneyback(True, -200)

150.0

In [147]:
lookup('12000019')

25    1610612739
Name: HOME_TEAM_ID, dtype: object


KeyError: 0

In [ ]:
money

In [ ]:
1610612737

In [110]:
teams["TEAM_ID"].unique()

array([1610612737, 1610612738, 1610612740, 1610612741, 1610612742,
       1610612743, 1610612745, 1610612746, 1610612747, 1610612748,
       1610612749, 1610612750, 1610612751, 1610612752, 1610612753,
       1610612754, 1610612755, 1610612756, 1610612757, 1610612758,
       1610612759, 1610612760, 1610612761, 1610612762, 1610612763,
       1610612764, 1610612765, 1610612766, 1610612739, 1610612744],
      dtype=int64)

In [142]:
#load data
teams = pd.read_csv('Data/teams.csv', dtype = {'TEAM_ID': str, 'HOME_TEAM_ID': str, 'VISITOR_TEAM_ID': str})
teams.head(3)

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate


In [ ]:
# TREE

In [52]:
target = final_data["HOME_TEAM_WINS"]
target_names = ["loss", "win"]

In [53]:
data_1 = test_data.drop("HOME_TEAM_WINS", axis=1)
feature_names = data_1.columns
data_1.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,FG_PCT_home_ROLLING,FT_PCT_home_ROLLING,FG3_PCT_home_ROLLING,AST_home_ROLLING,REB_home_ROLLING,FG_PCT_away_ROLLING,FT_PCT_away_ROLLING,...,FG_PCT_home_ALL,FT_PCT_home_ALL,FG3_PCT_home_ALL,AST_home_ALL,REB_home_ALL,FG_PCT_away_ALL,FT_PCT_away_ALL,FG3_PCT_away_ALL,AST_away_ALL,REB_away_ALL
269,12000001,1610612737,1610612753,0.4782,0.7872,0.4004,29.0,47.4,0.4396,0.8234,...,0.459219,0.771733,0.353589,25.781481,43.540741,0.446535,0.758665,0.343747,23.096654,42.479554
270,12000013,1610612737,1610612753,0.4492,0.8064,0.3656,26.6,49.2,0.4300,0.7804,...,0.458889,0.771945,0.353439,25.760148,43.605166,0.446574,0.758411,0.343533,23.107407,42.503704
567,12000041,1610612738,1610612751,0.4550,0.8282,0.3286,24.4,46.2,0.4264,0.7538,...,0.453574,0.788007,0.354641,24.963087,44.852349,0.445095,0.766844,0.341650,22.657795,43.992395
866,12000007,1610612739,1610612754,0.4784,0.7334,0.3590,28.0,44.2,0.5328,0.7874,...,0.467810,0.757083,0.371663,23.386667,43.793333,0.457529,0.778964,0.359598,22.807971,42.286232
867,12000019,1610612739,1610612754,0.4808,0.6884,0.3756,26.0,46.6,0.5200,0.7626,...,0.467897,0.756645,0.371458,23.375415,43.807309,0.457412,0.778679,0.359531,22.819495,42.270758


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_1, target, random_state=42)

In [54]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.3877551020408163

In [55]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

C:\Users\sjrya\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.3877551020408163

In [56]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.05488771248243362, 'FG_PCT_away_ALL'),
 (0.05370530650245657, 'AST_home_ALL'),
 (0.05318353065094898, 'FG_PCT_home_ALL'),
 (0.052434986231032425, 'FG3_PCT_home_ALL'),
 (0.05123418327922853, 'AST_away_ALL'),
 (0.05064076966892411, 'REB_home_ALL'),
 (0.05045160610767078, 'FG_PCT_home_ROLLING'),
 (0.049924047007827335, 'GAME_ID'),
 (0.04905406238359842, 'FT_PCT_away_ALL'),
 (0.0485038772448536, 'FT_PCT_home_ALL'),
 (0.04778010761974577, 'REB_away_ALL'),
 (0.04617125202488059, 'FG3_PCT_away_ALL'),
 (0.04608529611704485, 'FG_PCT_away_ROLLING'),
 (0.04602105744230488, 'FG3_PCT_home_ROLLING'),
 (0.04589032377916942, 'FT_PCT_home_ROLLING'),
 (0.04547451964866195, 'FT_PCT_away_ROLLING'),
 (0.04374944297516748, 'FG3_PCT_away_ROLLING'),
 (0.042218659216175576, 'AST_home_ROLLING'),
 (0.04153605291413814, 'REB_away_ROLLING'),
 (0.04108388921443893, 'REB_home_ROLLING'),
 (0.039969317489298016, 'AST_away_ROLLING'),
 (0.0, 'VISITOR_TEAM_ID'),
 (0.0, 'HOME_TEAM_ID')]

In [57]:
# ---------------- LEGACY CODE --------------------# 

In [58]:
# games['HOME_TEAM_ID'].unique()

In [59]:
# home_var = {'GAME_DATE_EST': [], 
#            'GAME_ID': [],
#            'HOME_TEAM_ID': [],
#            'FG_PCT_home': [],
#            'FT_PCT_home': [],
#            'FG3_PCT_home': [], 'AST_home': [], 'REB_home': [],'HOME_TEAM_WINS':[]
#            }


# home_data = pd.DataFrame(home_var)

# for team in games['HOME_TEAM_ID'].unique():

#     temp_df = games.loc[games['HOME_TEAM_ID']==team,:].sort_values(by='GAME_DATE_EST')

#     temp_df = temp_df[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'HOME_TEAM_WINS']]

#     temp_df['FG_PCTHOME_ROllING']=temp_df['FG_PCT_home'].shift().cumsum()
#     temp_df['FT_PCTHOME_ROllING']=temp_df['FT_PCT_home'].apply(lambda x: x.rolling(5, 1).mean().shift().bfill())
#     temp_df['FG3_PCTHOME_ROllING']=temp_df['FG3_PCT_home'].apply(lambda x: x.rolling(5, 1).mean().shift().bfill())
#     temp_df['AST_home_ROllING']=temp_df['AST_home'].apply(lambda x: x.rolling(5, 1).mean().shift().bfill())
#     temp_df['REB_home_ROllING']=temp_df['REB_home'].apply(lambda x: x.rolling(5, 1).mean().shift().bfill())

#     home_data = home_data.append(temp_df)
    
    
# home_data = home_data.dropna()
# home_data = home_data.drop(columns = ['FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home'])
# home_data.head()

In [60]:
# away_var = {'GAME_DATE_EST': [], 
#            'GAME_ID': [],
#            'VISITOR_TEAM_ID': [],
#            'FG_PCT_away': [],
#            'FT_PCT_away': [],
#            'FG3_PCT_away': [], 'AST_away': [], 'REB_away': []
#            }

# away_data = pd.DataFrame(away_var)

# for team in games['VISITOR_TEAM_ID'].unique():
    

#     temp_df = games.loc[games['VISITOR_TEAM_ID']==team,:].sort_values(by='GAME_DATE_EST')

#     temp_df = temp_df[['GAME_DATE_EST', 'GAME_ID', 'VISITOR_TEAM_ID', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away']]
    
#     temp_df['FG_PCTAWAY_ROllING']=temp_df['FG_PCT_away'].rolling(2).mean()
#     temp_df['FT_PCTAWAY_ROllING']=temp_df['FT_PCT_away'].rolling(2).mean()
#     temp_df['FG3_PCTAWAY_ROllING']=temp_df['FG3_PCT_away'].rolling(2).mean()
#     temp_df['AST_away_ROllING']=temp_df['AST_away'].rolling(2).mean()
#     temp_df['REB_away_ROllING']=temp_df['REB_away'].rolling(2).mean()

#     away_data = away_data.append(temp_df)

# away_data = away_data.dropna()
# away_data = away_data.drop(columns = ['FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away'])
# away_data.head()

In [61]:
# clean_data = pd.merge(home_data, away_data, how="inner", on=["GAME_DATE_EST", "GAME_ID"])

In [62]:
# clean_data = clean_data.drop(columns = ["GAME_DATE_EST"])
# clean_data.head()

In [63]:
# Logistical Regression

In [64]:
# # Assign X (data) and y (target)
# X = clean_data.drop("HOME_TEAM_WINS", axis=1)
# y = clean_data["HOME_TEAM_WINS"].values.reshape(-1, 1)
# print(X.shape, y.shape)

In [65]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


# X_train.head()

In [66]:
# # Create the scalers using only the training data!
# from sklearn.preprocessing import StandardScaler
# X_scaler = StandardScaler().fit(X_train)

In [67]:
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [68]:
# # create your model
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression()
# classifier

In [69]:
# # Fit (train) or model using the training data
# classifier.fit(X_train_scaled, y_train)

In [70]:
# # Validate the model using the .score() method
# print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

In [71]:
# inputs match up - home team / away team 
# query latest stats from api 
# latest rolling averages calculation
# retrain model 
# input user input - output hometeam wins

In [72]:
# XGBOOST

In [73]:
# # create your model
# model = XGBClassifier()

In [74]:
# model.fit(X_train_scaled, y_train)

In [75]:
# model.predict(X_test_scaled)

In [76]:
# y_pred= classifier.predict(X_test_scaled)

In [77]:
# from sklearn.metrics import accuracy_score

In [78]:
# accuracy_score(y_test, y_pred)

In [79]:
# TREE

In [80]:
# # Fit (train) using the training data
# model.fit(X_train_scaled, y_train)

In [81]:
# # Validate the model using the test data
# predictions = model.predict(X_test_scaled)

# accuracy = accuracy_score(y_test, predictions)
# print(accuracy)

In [82]:
# target = clean_data["HOME_TEAM_WINS"]
# target_names = ["loss", "win"]


In [83]:
# data_1 = clean_data.drop("HOME_TEAM_WINS", axis=1)
# feature_names = data_1.columns
# data_1.head()

In [84]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_1, target, random_state=42)

In [85]:
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X_train, y_train)
# clf.score(X_test, y_test)

In [86]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=200)
# rf = rf.fit(X_train, y_train)
# rf.score(X_test, y_test)

In [87]:
# sorted(zip(rf.feature_importances_, feature_names), reverse=True)